In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import image_dataset_from_directory, to_categorical
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras import layers, models


In [ ]:

def load_dataset(dir, image_size=(32, 32)):
    dataset = image_dataset_from_directory(dir, image_size=image_size)
    images, labels = [], []
    for x, y in dataset:
        images.append(x.numpy())
        labels.append(y.numpy())
    X = np.concatenate(images, axis=0).astype('float32') / 255.0
    Y = np.concatenate(labels, axis=0)
    Y = to_categorical(Y, 10)
    return X, Y

def resize_and_preprocess(X, batch_size=500):
    """Process images in batches to avoid OOM"""
    processed = []
    for i in range(0, len(X), batch_size):
        batch = X[i:i+batch_size]
        batch = tf.image.resize(batch, (224, 224))
        batch = preprocess_input(batch.numpy())
        processed.append(batch)
    return np.concatenate(processed, axis=0)

# Load data


In [ ]:
X, Y = load_dataset('/content/drive/MyDrive/cifar10/cifar10/train')

# Sample subset (use less data)
indices = np.random.choice(len(X), size=5000, replace=False)
X, Y = X[indices], Y[indices]

# Split
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

# Resize and preprocess in batches
xtrain = resize_and_preprocess(xtrain)
xtest = resize_and_preprocess(xtest)

# Build model
base_model = VGG19(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dense(10, activation="softmax")
])



In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train
history = model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=5, batch_size=16)

# Plot
plt.plot(history.history["accuracy"], label="Train")
plt.plot(history.history["val_accuracy"], label="Val")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
#alex
from keras_applications_contrib.alexnet import AlexNet
from tensorflow.keras import layers, models

base_model = AlexNet(include_top=False, input_shape=(227,227,3), weights=None) 
base_model.trainable = False

model = models.Sequential([
    layers.Resizing(227, 227),        
    layers.Rescaling(1./255),         
    base_model,
    layers.GlobalAveragePooling2D(),  
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax"),
])